# FaceReader representations
This notebook gets the FaceReader representations for all the syndromic and control images and saves them to two csv files.

In [ ]:
import clr
from os.path import isfile
import json
import glob
from os import listdir
from os.path import join, isfile
import numpy as np
import csv
from tqdm import tqdm
from collections.abc import Mapping

In [ ]:
import sys
sys.path.append("..")
from global_variables import GENERAL_DIR, syn_list, SDK_ref, license, LEFT, RIGHT

clr.AddReference(SDK_ref)
from VicarVision import FaceReader
clr.AddReference('System.Drawing')
from System import Drawing
FR = FaceReader.FaceReaderSDK(license)

Read in all syndromic image and save the landmark representation to one csv file.

In [ ]:
syn_rep = []

# for each syndrome
for syn_name in syn_list:
    syn_dir = GENERAL_DIR + "\\{}\\{}-all-photos".format(syn_name, syn_name)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f)) and ".jpg" in f)]
        
    # for every image of this syndrome
    for filename in tqdm(files_syn): 
        
        # open file
        FR.GrabCredits(1)        
        bitmap = Drawing.Bitmap(join(syn_dir, filename))
        try:
            result = (FR.AnalyzeFace(bitmap))
            result = json.loads(result.ToJson())
        except:
            print("An exception occurred")
            result = 'error'
            syn_rep.append([filename] + np.zeros(510*3).tolist())
            
        if isinstance(result, Mapping):
            if result['FaceAnalyzed']:
                landmarks = []
                landmarks_dict = result['Landmarks3D']
                for item in landmarks_dict:
                    landmarks.append(item['X'])
                    landmarks.append(item['Y'])
                    landmarks.append(item['Z'])
                syn_rep.append([filename] + landmarks)
            else:
                syn_rep.append([filename] + np.zeros(510*3).tolist())

Read in all control images and save the landmark representation to one csv file.

In [ ]:
ID_rep = []
ID_dir = GENERAL_DIR + "\\ID-controls"

# get list of filenames
files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f)) and ".jpg" in f)]

# for every image of this syndrome
for filename in files_ID: 

    # open file
    FR.GrabCredits(1)        
    bitmap = Drawing.Bitmap(join(ID_dir, filename))
    try:
        result = (FR.AnalyzeFace(bitmap))
        result = json.loads(result.ToJson())
    except:
        print("An exception occurred")
        result = 'error'
        ID_rep.append([filename] + np.zeros(510*3).tolist())

    if isinstance(result, Mapping):
        if result['FaceAnalyzed']:
            landmarks = []
            landmarks_dict = result['Landmarks3D']
            for item in landmarks_dict:
                landmarks.append(item['X'])
                landmarks.append(item['Y'])
                landmarks.append(item['Z'])
            ID_rep.append([filename] + landmarks)
        else:
            ID_rep.append([filename] + np.zeros(510*3).tolist())